In [ ]:
%%capture

%pip install --upgrade pip 
%pip install datasets[audio]
%pip install evaluate
%pip install transformers
%pip install jiwer
%pip install accelerate
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset

shn_dataset_train = load_dataset("norhsangpha/shn-asr-datasets", split="train", token=True)
shn_dataset_test = load_dataset("norhsangpha/shn-asr-datasets", split="test", token=True)


In [6]:
from datasets import load_from_disk

shn_dataset_train = load_from_disk("../../Datasets/mms-asr-nova-datasets-500/train")
shn_dataset_test = load_from_disk("../../Datasets/mms-asr-nova-datasets-500/test")

In [8]:
print(shn_dataset_train)
print(shn_dataset_test)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 422
})
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 105
})


In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(shn_dataset_train.remove_columns(["audio"]), num_examples=10)

,transcription
0,ၶဝ်ၵေႃႈ ဢွၼ်ၵၼ်ဢွၵ်ႇသုမ်ႉမႃးတူၺ်းသေ လႆႈႁၼ်ၽူႈမၢႆၼၼ်ႉ ၼွၼ်းလူမ်ႉတၢႆၵႂႃႇယဝ်ႉ
1,ၶႃႈတေသိုပ်ႇဢဝ်ပုၼ်ႈၽွၼ်းၵႂႃႇၵူၺ်း လူဝ်ႇႁဵတ်းသင်ၵေႃႈဢုပ်ႇၵၼ်တင်းဝၼ်းၸိုၼ်ႈၵႂႃႇလႄႈၼႃႈ
2,သမ်ႉဝႃႈလွင်ႈမၼ်းပူၼ်ႉမႃးႁိုင်တၢၼ်ႇႁိုဝ်ၵေႃႈ လွင်ႈဝၼ်းလင်ၼၼ်ႉ ယင်းတိုၵ်ႉဢဝ်မႃးလၢတ်ႈထိုင်ၵၼ်လႆႈယူႇ
3,ဝၼ်းၸိုၼ်ႈယၵ်ႉတူၺ်းၼႃႈၸၢႆးလိူၼ်ယဝ်ႉ ထူၺ်ႈၸႂ်ဢွၵ်ႇမႃးၼင်ႇလူမ်ၸႂ်
4,ၵူၼ်းၼႆႉ မၼ်းတေမီးၽႂ်ဢၼ်ႁၼ်ငိုၼ်းယဝ်ႉ တေထဵင်လႆႈ ၼႂ်းၼၼ်ႉၵေႃႈယင်းပႃးၵူၼ်းၸိူင်ႉၼင်ႇၸဝ်ႈမိူင်းၶမ်းယူႇလူး
5,ဝၼ်းၸိုၼ်ႈဢဝ်လႅၼ်းယႃႈမၼ်းထႅင်ႈလႅၼ်းၼိုင်ႈ ၸပ်းၾႆးယဝ်ႉ လုတ်ႇၵႂႃႇၵမ်းၼိုင်ႈသေလၢတ်ႈထႅင်ႈ
6,မိူဝ်ႈပိူၼ်ႈၵႂႃႇမူတ်းယဝ်ႉ ၸဝ်ႈလွၼ်ႉၶိူဝ်းၼၼ်ႉၵေႃႈ မႆႈၸႂ်မႆႈၶေႃး သမ်ႉပေႃးၵွတ်ႇၶမ်းလိုၼ်းၶဝ်ႈယူႇၼႂ်းဢူၵ်းမၼ်းၵႂႃႇယဝ်ႉ
7,ပေႃးဝႃႈပဵၼ်ၵၢၼ်လႆႈငိုၼ်း သမ်ႉဝႃႈမၼ်းယၢပ်ႇၵေႃႈ ၵႃႈတေလႆႈၸုၼ်ႉႁေႁဵတ်းဢိူဝ်ႈ
8,သွင်ၶႃၵေႃႈဢမ်ႇသိုပ်ႇလၢတ်ႈသင်ထႅင်ႈ လိူဝ်သေပဝ်ႇပုၼ်ႈၵႂၼ်းလႅၼ်းယႃႈၶႃၵႂႃႇ ႁင်းၽႂ်ႁင်းမၼ်းၵူၺ်း
9,တႃႇမႃးၸူးၶမ်းလိုၼ်းသေ ၵႄႈၶႆလွင်ႈၼႆႉၵႂႃႇႁႂ်ႈယဝ်ႉတူဝ်ႈၼႆယူႇ


In [10]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\။\၊\…]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

shn_dataset_train = shn_dataset_train.map(remove_special_characters)
shn_dataset_test = shn_dataset_test.map(remove_special_characters)

Map:   0%|          | 0/422 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

In [11]:
show_random_elements(shn_dataset_train.remove_columns(["audio"]))

,transcription
0,မႂ်းႁူႉလွင်ႈၵဝ်ၼမ်ၼႃႇယဝ်ႉၼႃႈ ဢၼ်တင်ႈၸႂ်လၵ်ႉၶူဝ်ၶွင်ပိူၼ်ႈသေ တင်ႈၸႂ်ႁႂ်ႈပိူၼ်ႈၺွပ်းလႆႈတိၵ်းတိၵ်းၼႆႉ
1,ၼႃႉတေလၢတ်ႈၵႂၢမ်းလပ်ႉၼႃ မိူဝ်ႈၸၢႆးႁွင်ႉၽႂ်ဝႃႈဢူႈ ၵေႃႉၼၼ်ႉၵေႃႈပဵၼ်ဢူႈၸၢႆးယဝ်ႉၼႃႈ
2,ၵၢၼ်မၼ်းၼၼ်ႉ ဢမ်ႇတွၼ်ႉမီးမိူဝ်ႈယဝ်ႉႁေပွၵ်ႈလႄႈ ပေႃးဢၼ်ၼိုင်ႈတၢၼ်ႇယဝ်ႉၵႂႃႇ ထႅင်ႈဢၼ်ၵေႃႈသမ်ႉပေႃးမႃးယူႇယဝ်ႉ
3,ၼႂ်းၶၢဝ်းတၢင်းသွင်သၢမ်ပီမႃးၼႆႉ ၸဝ်ႈလွၼ်ႉၶိူဝ်းဢမ်ႇပေႃးတၼ်းလႄႈ ဢမ်ႇပေႃးသူႈလႆႈမႃးၸႂ်းၶဝ်မႄႈလုၵ်ႈ
4,ၼႂ်းလိၵ်ႈၼၼ်ႉတႄႉ လႆႈတႅမ်ႈထိုင် ၶေႃႈၵႂၢမ်းၼႂ်းဢူၵ်းႁူဝ်ၸႂ် ဢၼ်လႆႈႁၵ်ႉမႅၼ်ႈဝၼ်းၸိုၼ်ႈလူၼ်ႉလိူဝ်ႁႅင်း
5,ဢၢႆႈၸၢႆး ၸွင်ႇမႂ်းႁူႉႁိုဝ်ဝႃႈ ပေႃးမီးၼႃႈႁိူၼ်းၼႆ ၼႃႈႁိူၼ်းၼၼ်ႉ ပဵၼ်တီႈယူပ်ႈယွမ်းႁဝ်းမႃးၵမ်းလဵဝ်
6,ဢမ်ႇပိူင်ႈသင်ၵၼ်တင်းဢၢႆႈၸိူဝ်းၼၼ်ႉၼင်ႇၵဝ်ႇဢမ်ႇၸႂ်ႈႁိုဝ် ဝူၼ်ႉဝႃႈဢၼ်သူႁဵတ်းၵႂႃႇၼၼ်ႉ မၼ်းၵိုင်ႇၵႃႈမၼ်းယူႇႁႃႉ
7,ႁွင်ႉလၢတ်ႈၸူးဝၼ်းၸိုၼ်ႈၵႂႃႇ တွင်းပၢၼ်ႇယဝ်ႉၶႃႈ ၶႃႈၵူဝ်ယဝ်ႉ
8,လွင်ႈၼႆႉၵေႃႈတေၸၢင်ႈၸွႆႈမွတ်ႇပႅတ်ႈ လွင်ႈဢၼ်မၼ်းလႆႈႁဵတ်းၽိတ်းၵႂႃႇဝၼ်းၼၼ်ႉၼႆယူႇ
9,ဢေႃႈ ၶႃႈတေဢဝ်ၵႂႃႇပၼ်ၶဝ်ၶိုၼ်းတင်းမူတ်း ဢၼ်ၶႃႈလၵ်ႉမႃးၼၼ်ႉ


In [12]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = shn_dataset_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=shn_dataset_train.column_names)
vocab_test = shn_dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=shn_dataset_test.column_names)

Map:   0%|          | 0/422 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

In [13]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [14]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

vocab_dict

{' ': 0,
 '\xa0': 1,
 'င': 2,
 'တ': 3,
 'ထ': 4,
 'ပ': 5,
 'မ': 6,
 'ယ': 7,
 'လ': 8,
 'ဝ': 9,
 'သ': 10,
 'ဢ': 11,
 'ိ': 12,
 'ီ': 13,
 'ု': 14,
 'ူ': 15,
 'ေ': 16,
 'ဵ': 17,
 'း': 18,
 '်': 19,
 'ျ': 20,
 'ြ': 21,
 'ွ': 22,
 'ၢ': 23,
 'ၵ': 24,
 'ၶ': 25,
 'ၸ': 26,
 'ၺ': 27,
 'ၼ': 28,
 'ၽ': 29,
 'ၾ': 30,
 'ႁ': 31,
 'ႂ': 32,
 'ႃ': 33,
 'ႄ': 34,
 'ႅ': 35,
 'ႆ': 36,
 'ႇ': 37,
 'ႈ': 38,
 'ႉ': 39}

In [15]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [16]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)


42

In [17]:
target_lang = "shn"
# new_vocab_dict = {target_lang: vocab_dict}

In [18]:
from transformers import Wav2Vec2CTCTokenizer

mms_adapter_repo = "facebook/mms-1b-all"

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(mms_adapter_repo)
new_vocab = tokenizer.vocab

new_vocab[target_lang] = vocab_dict

In [19]:
import json
with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False)


In [20]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [36]:
repo_name = "./models/wav2vec2-large-mms-1b-shan"
# tokenizer.push_to_hub(repo_name)


In [21]:
tokenizer.save_pretrained("./models/wav2vec2-large-mms-1b-shan")

('./models/wav2vec2-large-mms-1b-shan\\tokenizer_config.json',
 './models/wav2vec2-large-mms-1b-shan\\special_tokens_map.json',
 './models/wav2vec2-large-mms-1b-shan\\vocab.json',
 './models/wav2vec2-large-mms-1b-shan\\added_tokens.json')

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [24]:
shn_dataset_train[0]["audio"]

{'path': 'audio10.wav',
 'array': array([-1.06073006e-09,  1.29704758e-09, -1.43051804e-09, ...,
         7.50327745e-05,  6.82075042e-05,  0.00000000e+00]),
 'sampling_rate': 22050}

In [25]:
from datasets import Audio

shn_dataset_train = shn_dataset_train.cast_column("audio", Audio(sampling_rate=16_000))
shn_dataset_test = shn_dataset_test.cast_column("audio", Audio(sampling_rate=16_000))


In [26]:
shn_dataset_train[0]["audio"]

{'path': 'audio10.wav',
 'array': array([ 1.44173100e-08, -1.53559085e-08,  1.59961928e-08, ...,
         3.89381676e-05,  7.58816605e-05,  0.00000000e+00]),
 'sampling_rate': 16000}

In [27]:
rand_int = random.randint(0, len(shn_dataset_train)-1)

print("Target text:", shn_dataset_train[rand_int]["transcription"])
print("Input array shape:", shn_dataset_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", shn_dataset_train[rand_int]["audio"]["sampling_rate"])


Target text: ၵူၺ်းၵႃႈ သူဝူၼ်ႉဝႃႈ ပိူၼ်ႈတေပွႆႇသူလွတ်ႈလႆႈယူႇႁႃႉ
Input array shape: (73392,)
Sampling rate: 16000


## Dataset Prepair

In [28]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["transcription"]).input_ids
    return batch


In [29]:
shn_dataset_train = shn_dataset_train.map(prepare_dataset, remove_columns=shn_dataset_train.column_names)
shn_dataset_test = shn_dataset_test.map(prepare_dataset, remove_columns=shn_dataset_test.column_names)

Map:   0%|          | 0/422 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

## Training

In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

#### Evaluate

In [31]:
from evaluate import load

wer_metric = load("wer")

In [32]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/mms-1b-all",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)


Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

In [34]:
model.init_adapter_layers()
model.freeze_base_model()

adapter_weights = model._get_adapters()
for param in adapter_weights.values():
    param.requires_grad = True


In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=200,
  eval_steps=100,
  logging_steps=5,
  learning_rate=1e-3,
  warmup_steps=100,
  save_total_limit=2,
  push_to_hub=False,
)

c:\Users\NoerN\.conda\envs\mmslabs\lib\site-packages\transformers\training_args.py:1483: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=shn_dataset_train,
    eval_dataset=shn_dataset_test,
    tokenizer=processor.feature_extractor,
)

In [39]:
trainer.train()

  0%|          | 0/140 [00:00<?, ?it/s]

c:\Users\NoerN\.conda\envs\mmslabs\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
c:\Users\NoerN\.conda\envs\mmslabs\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\NoerN\.conda\envs\mmslabs\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:963: UserWarning: 1Torch was not compiled with flash at

{'loss': 10.5366, 'grad_norm': 22.734519958496094, 'learning_rate': 3e-05, 'epoch': 0.36}
{'loss': 10.3854, 'grad_norm': 25.92425537109375, 'learning_rate': 8e-05, 'epoch': 0.71}
{'loss': 9.2912, 'grad_norm': 26.86384391784668, 'learning_rate': 0.00013000000000000002, 'epoch': 1.07}
{'loss': 7.9499, 'grad_norm': 30.73958396911621, 'learning_rate': 0.00017999999999999998, 'epoch': 1.43}
{'loss': 6.3229, 'grad_norm': 28.897329330444336, 'learning_rate': 0.00023, 'epoch': 1.79}
{'loss': 4.5586, 'grad_norm': 11.591548919677734, 'learning_rate': 0.00028000000000000003, 'epoch': 2.14}
{'loss': 3.5702, 'grad_norm': 9.040257453918457, 'learning_rate': 0.00033, 'epoch': 2.5}
{'loss': 3.4151, 'grad_norm': 3.3571383953094482, 'learning_rate': 0.00038, 'epoch': 2.86}
{'loss': 3.1079, 'grad_norm': 4.979417324066162, 'learning_rate': 0.00043, 'epoch': 3.21}
{'loss': 2.789, 'grad_norm': 2.4046664237976074, 'learning_rate': 0.00048, 'epoch': 3.57}
{'loss': 2.223, 'grad_norm': 1.9828133583068848, 'lear

  0%|          | 0/14 [00:00<?, ?it/s]

{'eval_loss': 0.2099316567182541, 'eval_wer': 0.5904761904761905, 'eval_runtime': 4.0181, 'eval_samples_per_second': 26.131, 'eval_steps_per_second': 3.484, 'epoch': 7.14}
{'loss': 0.2726, 'grad_norm': 0.364082932472229, 'learning_rate': 0.000925, 'epoch': 7.5}
{'loss': 0.2404, 'grad_norm': 0.496386855840683, 'learning_rate': 0.0008, 'epoch': 7.86}
{'loss': 0.235, 'grad_norm': 0.614963173866272, 'learning_rate': 0.000675, 'epoch': 8.21}
{'loss': 0.2115, 'grad_norm': 0.589288592338562, 'learning_rate': 0.00055, 'epoch': 8.57}
{'loss': 0.2134, 'grad_norm': 0.5161349773406982, 'learning_rate': 0.000425, 'epoch': 8.93}
{'loss': 0.2422, 'grad_norm': 0.6543455123901367, 'learning_rate': 0.0003, 'epoch': 9.29}
{'loss': 0.204, 'grad_norm': 0.555431604385376, 'learning_rate': 0.000175, 'epoch': 9.64}
{'loss': 0.218, 'grad_norm': 0.8375492095947266, 'learning_rate': 5e-05, 'epoch': 10.0}
{'train_runtime': 505.0041, 'train_samples_per_second': 8.356, 'train_steps_per_second': 0.277, 'train_loss':

TrainOutput(global_step=140, training_loss=2.5615255756037576, metrics={'train_runtime': 505.0041, 'train_samples_per_second': 8.356, 'train_steps_per_second': 0.277, 'total_flos': 2.8245716024730936e+18, 'train_loss': 2.5615255756037576, 'epoch': 10.0})

In [40]:
from safetensors.torch import save_file as safe_save_file
from transformers.models.wav2vec2.modeling_wav2vec2 import WAV2VEC2_ADAPTER_SAFE_FILE
import os

adapter_file = WAV2VEC2_ADAPTER_SAFE_FILE.format(target_lang)
adapter_file = os.path.join(training_args.output_dir, adapter_file)

safe_save_file(model._get_adapters(), adapter_file, metadata={"format": "pt"})


In [46]:
trainer.save_model("./models/wav2vec2-large-mms-1b-shan")

In [ ]:
trainer.push_to_hub()